In [1]:
import os 

os.chdir("app/")

In [2]:
from functools import partial

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.trainer import Trainer

from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.data_load.datasets import MemoryMapDataset
from ptls.preprocessing import PandasDataPreprocessor

from ptls.nn import TrxEncoder

from ptls.frames import PtlsDataModule

from nn.seq_encoder import ConvSeqEncoder
from modules.ts2vec_module import TS2Vec
from datasets.datasets import TS2VecDataset
from utils.encode import encode_data
from utils.evaluation import bootstrap_eval

## Load data

In [3]:
df = pd.read_parquet("data/preprocessed_new/default_date.parquet")
df.head()

,user_id,mcc_code,amount,timestamp,holiday_target,weekend_target,global_target,default_target,time_delta
0,69,5,-342.89792,2021-03-05 02:52:36,0,0,0,0,0
1,69,21,-1251.88120,2021-03-05 09:43:28,0,0,0,0,24652
2,69,12,-87.30924,2021-03-05 11:17:23,0,0,0,0,5635
3,69,6,-1822.17700,2021-03-05 13:41:03,0,0,0,0,8620
4,69,18,-427.12363,2021-03-05 19:14:23,0,0,0,0,20000


## Prepare dataset and dataloaders

In [4]:
preprocessor = PandasDataPreprocessor(
    col_id="user_id",
    col_event_time="timestamp",
    event_time_transformation="dt_to_timestamp",
    cols_category=["mcc_code"],
    cols_first_item=["global_target"]
)

In [5]:
data = preprocessor.fit_transform(df)

In [6]:
y = np.array([data[i]["global_target"] for i in range(len(data))])

In [7]:
val_size = 0.1
test_size = 0.1

train, val_test, y_train, y_val_test = train_test_split(data, y, stratify=y, test_size=test_size+val_size, random_state=42)
val, test, y_val, y_test = train_test_split(val_test, y_val_test, stratify=y_val_test, test_size=test_size/(test_size+val_size), random_state=42)

In [8]:
# oversampling

np.random.seed(42)

pos_num = y_train.sum()
neg_num = len(y_train) - pos_num

pos_idx = np.arange(len(y_train))[y_train == 1]

pos_idx_os = np.random.choice(pos_idx, size=(neg_num-pos_num), replace=True)

train_os = train + [train[idx] for idx in pos_idx_os]

In [9]:
train_ds = TS2VecDataset(train_os, min_seq_len=15)
val_ds = TS2VecDataset(val, min_seq_len=15)
test_ds = MemoryMapDataset(test, [SeqLenFilter(min_seq_len=15)])

In [10]:
datamodule = PtlsDataModule(
    train_data=train_ds,
    valid_data=val_ds,
    train_batch_size=128,
    valid_batch_size=128,
    train_num_workers=8,
    valid_num_workers=8
)

# TS2Vec

## Model training

In [11]:
trx_encoder = TrxEncoder(
    embeddings={
        "mcc_code": {"in": 345, "out": 16}
    },
    numeric_values={
        "amount": "identity",
    },
    use_batch_norm_with_lens=True,
    norm_embeddings=False,
    embeddings_noise=0.003
)

seq_encoder = ConvSeqEncoder(
    trx_encoder,
    hidden_size=800,
    num_layers=10,
    dropout=0.1,
)

In [12]:
lr_scheduler_partial = partial(torch.optim.lr_scheduler.ReduceLROnPlateau, factor=.9025, patience=5, mode="min")
optimizer_partial = partial(torch.optim.Adam, lr=4e-3)

model = TS2Vec(
    seq_encoder,
    optimizer_partial=optimizer_partial,
    lr_scheduler_partial=lr_scheduler_partial
)

In [13]:
early_stopping = EarlyStopping(
    monitor="valid_loss", 
    mode="min",
    patience=5
)

checkpoint = ModelCheckpoint(
    monitor="valid_loss", 
    mode="min"
)

trainer = Trainer(
    max_epochs=25,
    devices=[0],
    accelerator="gpu",
    callbacks=[checkpoint, early_stopping]
)

trainer.fit(model, datamodule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name         | Type                        | Params
-------------------------------------------------------------
0 | _loss        | HierarchicalContrastiveLoss | 0     
1 | _seq_encoder | ConvSeqEncoder              | 2.0 M 
2 | _head        | Head                        | 0     
3 | valid_loss   | MeanMetric                  | 0     
-------------------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/macro-micro-coles/miniconda/envs/env/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [14]:
model.load_state_dict(torch.load(checkpoint.best_model_path)["state_dict"])

<All keys matched successfully>

In [39]:
model.load_state_dict(torch.load("lightning_logs/version_85/checkpoints/epoch=47-step=2160.ckpt")["state_dict"])

<All keys matched successfully>

In [26]:
torch.save(model.seq_encoder.state_dict(), "ts2vec_default.pth")

## Model evaluation

In [15]:
train_val_ds = MemoryMapDataset(train + val, [SeqLenFilter(min_seq_len=15)])

In [16]:
X_train, y_train = encode_data(model.seq_encoder, train_val_ds)
X_test, y_test = encode_data(model.seq_encoder, test_ds)

print("Train size:", len(y_train))
print("Test size:", len(y_test))

Train size: 6372
Test size: 708


In [19]:
res = bootstrap_eval(X_train, X_test, y_train, y_test)

100%|██████████| 10/10 [00:34<00:00,  3.50s/it]


In [20]:
res

,ROC-AUC,PR-AUC,Accuracy
0,0.544834,0.039652,0.963277
1,0.596661,0.072166,0.963277
2,0.600383,0.055926,0.963277
3,0.575344,0.054233,0.963277
4,0.607151,0.054302,0.963277
5,0.554252,0.051951,0.963277
6,0.603203,0.054514,0.963277
7,0.571509,0.051729,0.963277
8,0.612396,0.067901,0.963277
9,0.608504,0.055513,0.963277


In [21]:
res.agg(["mean", "std"])

,ROC-AUC,PR-AUC,Accuracy
mean,0.587424,0.055789,0.963277
std,0.024205,0.008900,0.000000


In [17]:
from sklearn.metrics import roc_auc_score, average_precision_score
from lightgbm import LGBMClassifier


lgbm = LGBMClassifier(
    n_estimators=500,
    boosting_type="gbdt",
    subsample=0.5,
    subsample_freq=1,
    learning_rate=0.02,
    feature_fraction=0.75,
    max_depth=6,
    lambda_l1=1,
    lambda_l2=1,
    min_data_in_leaf=50,
    random_state=42,
    n_jobs=8,
    verbose=-1
)

lgbm.fit(X_train, y_train)

LGBMClassifier(feature_fraction=0.75, lambda_l1=1, lambda_l2=1,
               learning_rate=0.02, max_depth=6, min_data_in_leaf=50,
               n_estimators=500, n_jobs=8, random_state=42, subsample=0.5,
               subsample_freq=1, verbose=-1)

In [18]:
y_pred = lgbm.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred), average_precision_score(y_test, y_pred)

(0.6361944507105798, 0.05592283362602761)

# TS2Vec with time features

## Model training

In [27]:
trx_encoder = TrxEncoder(
    embeddings={
        "mcc_code": {"in": 345, "out": 24}
    },
    numeric_values={
        "amount": "identity",
        "event_time": "identity",
        "time_delta": "identity",
    },
    use_batch_norm_with_lens=True,
    norm_embeddings=False,
    embeddings_noise=0.0003
)

seq_encoder = ConvSeqEncoder(
    trx_encoder,
    hidden_size=1024,
    num_layers=10,
    dropout=0.1,
)

In [28]:
lr_scheduler_partial = partial(torch.optim.lr_scheduler.ReduceLROnPlateau, factor=.9025, patience=5, mode="min")
optimizer_partial = partial(torch.optim.Adam, lr=4e-3)

model = TS2Vec(
    seq_encoder,
    optimizer_partial=optimizer_partial,
    lr_scheduler_partial=lr_scheduler_partial
)

In [29]:
checkpoint = ModelCheckpoint(
    monitor="valid_loss", 
    mode="min"
)

trainer = Trainer(
    max_epochs=50,
    devices=[1],
    accelerator="gpu",
    callbacks=[checkpoint]
)

trainer.fit(model, datamodule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name         | Type                        | Params
-------------------------------------------------------------
0 | _loss        | HierarchicalContrastiveLoss | 0     
1 | _seq_encoder | ConvSeqEncoder              | 3.3 M 
2 | _head        | Head                        | 0     
3 | valid_loss   | MeanMetric                  | 0     
-------------------------------------------------------------
3.3 M     Trainable params
0         Non-trainable params
3.3 M     Total params
13.248    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/macro-micro-coles/miniconda/envs/env/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1933: PossibleUserWarning: The number of training batches (45) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [34]:
model.load_state_dict(torch.load(checkpoint.best_model_path)["state_dict"])

<All keys matched successfully>

In [30]:
torch.save(model.seq_encoder.state_dict(), "ts2vec_default_date.pth")

## Model evaluation

In [35]:
train_val_ds = MemoryMapDataset(train + val, [SeqLenFilter(min_seq_len=15)])

In [36]:
X_train, y_train = encode_data(model.seq_encoder, train_val_ds)
X_test, y_test = encode_data(model.seq_encoder, test_ds)

print("Train size:", len(y_train))
print("Test size:", len(y_test))

Train size: 6372
Test size: 708


In [37]:
from sklearn.metrics import roc_auc_score, average_precision_score
from lightgbm import LGBMClassifier


lgbm = LGBMClassifier(
    n_estimators=500,
    boosting_type="gbdt",
    subsample=0.5,
    subsample_freq=1,
    learning_rate=0.02,
    feature_fraction=0.75,
    max_depth=6,
    lambda_l1=1,
    lambda_l2=1,
    min_data_in_leaf=50,
    random_state=42,
    n_jobs=8,
    verbose=-1
)

lgbm.fit(X_train, y_train)

LGBMClassifier(feature_fraction=0.75, lambda_l1=1, lambda_l2=1,
               learning_rate=0.02, max_depth=6, min_data_in_leaf=50,
               n_estimators=500, n_jobs=8, random_state=42, subsample=0.5,
               subsample_freq=1, verbose=-1)

In [38]:
y_pred = lgbm.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred), average_precision_score(y_test, y_pred)

(0.5350214301827205, 0.06420740187863244)